In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("..")

import numpy as np
import matplotlib.pyplot as plt
import os
from src import data
import json
from tqdm.auto import tqdm
from src.metrics import AggregateMetric

from src.utils.sweep_utils import read_sweep_results, relation_from_dict
import logging
from src.utils import logging_utils
from src import hparams

# logger = logging.getLogger(__name__)

# logging.basicConfig(
#     level=logging.DEBUG,
#     format = logging_utils.DEFAULT_FORMAT,
#     datefmt=logging_utils.DEFAULT_DATEFMT,
#     stream=sys.stdout
# )

In [ ]:
############################################
sweep_root = "../results/sweep-24-trials"
model_name = "gptj"
############################################

sweep_path = f"{sweep_root}/{model_name}"

In [ ]:
sweep_results = read_sweep_results(
    sweep_path, 
    # relation_names=["country capital city"]
)
list(sweep_results.keys())

In [ ]:
relation_dict = {}
for relation_name, sweep_dict in tqdm(sweep_results.items()):
    relation_dict[relation_name] = relation_from_dict(sweep_dict)
    

In [ ]:
for relation_name, relation in relation_dict.items():
    best_hparams = relation.best_by_efficacy(beta=2.5)
    performance = f"efficacy={best_hparams.efficacy.mean:.3f} | faithfulness={best_hparams.recall.mean:.3f}"
    print(f"{relation_name} >> layer={best_hparams.layer} | beta={best_hparams.beta.mean} | rank={int(best_hparams.rank.mean)} <> {performance}")

    hparams.RelationHParams(
        relation_name=relation.relation_name,
        h_layer=best_hparams.layer,
        h_layer_edit=best_hparams.layer,
        z_layer=-1,
        beta=best_hparams.beta.mean,
        rank=int(np.floor(best_hparams.rank.mean)),
        model_name=model_name,
    ).save()

In [ ]:
for relation_name, relation in relation_dict.items():
    best_hparams = relation.best_by_faithfulness(beta=2.5)
    performance = f"efficacy={best_hparams.efficacy.mean:.3f} | faithfulness={best_hparams.recall.mean:.3f}"
    print(f"{relation_name} >> layer={best_hparams.layer} | beta={best_hparams.beta.mean} | rank={int(best_hparams.rank.mean)} <> {performance}")

    hparams.RelationHParams(
        relation_name=relation.relation_name,
        h_layer=best_hparams.layer,
        h_layer_edit=best_hparams.layer,
        z_layer=-1,
        beta=best_hparams.beta.mean,
        rank=int(np.floor(best_hparams.rank.mean)),
        model_name=model_name,
    ).save()